In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cmath

In [ ]:
x = np.loadtxt('Data/Xtrainx.csv', delimiter=',') #load data for sigmax 
labels = np.loadtxt('Data/Ytrain.csv', delimiter=',') #load labels

In [ ]:
#The original trajectories contain 20001 time points. At t=0 and t = 0.0005 all of the signals are equal to 1, so 
#we omit the first time step.
X = x[:, 1:x.shape[1]] 

In [ ]:
#A function to perform the non-uniform discrete fourier transform. arr is an array containing the time signal, t 
#is an array containing the associated times
def nudft(arr, t):
    
    N = len(arr)
    
    #scaling the times to fall between 0 and 1
    tscaled = (t - np.amin(t))/(np.amax(t) - np.amin(t))
    
    #an array to store the Fourier coefficients
    coeff = np.zeros(len(arr), dtype=complex)
    
    #Generating an array of frequencies
    freq = np.arange(0, N, 1)
    
    #calculating the Fourier coefficients
    for i in range(len(freq)):
        for j in range(len(arr)):
            coeff[i] += arr[j]*cmath.exp(-2*np.pi*1j*freq[i]*tscaled[j])
            
    return(coeff)

In [ ]:
#function to take X and Y and return a training, test and validation set containing the Fourier coefficients
#obtained using the non-uniform discrete Fourier transform, along with the corresponding labels. times is an array
#of times corresponding to the trajectories in the training, validation and test sets.
def nufouriertrainvaltest(X, Y, times):
    
    #generating a training validation and test set.
    Xtrain = X[0:X.shape[0]-600, :]
    Xval = X[X.shape[0]-600:X.shape[0]-300, :]
    Xtest = X[X.shape[0]-300:X.shape[0], :]
    
    #The last 3 columns of Y contain the values of $'\eta'$, $\omega_c$ and $s$ so we only use the first three 
    #columns.
    Ytrain = Y[0:Y.shape[0]-600, 0:3]
    Yval = Y[Y.shape[0]-600:Y.shape[0]-300, 0:3]
    Ytest = Y[Y.shape[0]-300:Y.shape[0], 0:3]
    
    #Defining arrays to store the Fourier coefficients
    XtrainF = np.zeros_like(Xtrain, dtype=complex)
    XvalF = np.zeros_like(Xval, dtype = complex)
    XtestF = np.zeros_like(Xtest, dtype=complex)
    
    #Calculating the Fourier coefficients
    for i in range(Xtrain.shape[0]):
        XtrainF[i,:] = nudft(Xtrain[i,:], times)
        
    for i in range(Xtest.shape[0]):
        XvalF[i,:] = nudft(Xval[i,:], times)
        XtestF[i,:] = nudft(Xtest[i,:], times)
    
    #Splitting the coefficients up into their real and imaginary components
    xtrain = np.zeros((XtrainF.shape[0], 2*XtrainF.shape[1]))
    xval = np.zeros((XvalF.shape[0], 2*XvalF.shape[1]))
    xtest = np.zeros((XtestF.shape[0], 2*XtestF.shape[1]))

    for i in range(XtrainF.shape[0]):
        for j in range(XtrainF.shape[1]):
            xtrain[i, 2*j] = XtrainF[i,j].real
            xtrain[i, 2*j + 1] = XtrainF[i,j].imag
        
        
    for i in range(XtestF.shape[0]):
        for j in range(XtestF.shape[1]):
            xval[i, 2*j] = XvalF[i,j].real
            xval[i, 2*j + 1] = XvalF[i,j].imag
        
            xtest[i, 2*j] = XtestF[i,j].real
            xtest[i, 2*j + 1] = XtestF[i,j].imag
    
    return(xtrain, xval, xtest, Ytrain, Yval, Ytest)

In [ ]:
Xtrain = X[0:X.shape[0]-600, :]

#defining the correlation matrix
corrcoeff = np.zeros((Xtrain.shape[1], Xtrain.shape[1]))

for i in range(Xtrain.shape[1]):
    for j in range(Xtrain.shape[1]):
        #calculating the Pearson correlation coefficient between the ith and jth time point
        corrcoeff[i, j] = np.corrcoef(Xtrain[:,i], Xtrain[:,j])[0,1]

In [ ]:
#A function to sort the elements of an array into descending order and return the sorted values in a list, along 
#with the corresponding indices. 
def sortmatrix(arr):
    #an array to store the indices that would store each row in descending order
    sortedindicesrows = np.zeros((arr.shape[0], arr.shape[1]), dtype=int) 
    #an array to store the corresponding entries
    sortedvalsrows = np.zeros((arr.shape[0], arr.shape[1])) 
    
    for i in range(arr.shape[0]):
        #finding the indices that would sort each row in descending order
        sortedindicesrows[i,:] = (-arr[i,:]).argsort() 
        #finding the corresponding elements
        sortedvalsrows[i,:] = arr[i, sortedindicesrows[i,:]] 
    
    #flattening the array to apply argsort
    sortedrowsflat = sortedvalsrows.flatten() 
    #finding the indices that would sort the flattened array in descending order
    indicesflat = (-sortedrowsflat).argsort() 
    
    #defining a list to sore tuples of indices
    sortedindices = [] 
    #defining a list to store the sorted values corresponding to the indices
    sortedvals = [] 
    
    #finding the row and column indices in the original array
    for i in range(len(indicesflat)):
        for j in range(arr.shape[0]):
            if(j*arr.shape[1]) <= indicesflat[i] <= ((j+1)*arr.shape[1]-1):
                rowindex = j #row index
                columnindex = sortedindicesrows[j, indicesflat[i]-j*arr.shape[1]] #column index
                
                sortedindices.append((int(j), int(columnindex))) #adding tuple to the list of indices
                sortedvals.append(arr[int(j), int(columnindex)]) #adding the corresponding element in the array
    return(sortedindices, sortedvals)

In [ ]:
#Sorting the elements in the correlation matrix
sortedindices, sortedvals = sortmatrix(np.abs(corrcoeff))

In [ ]:
Ytrain = labels[0:labels.shape[0]-600, 0:3]

#Finding the trajectories in the training set which correspond to Ohmic, sub-Ohmic and super-Ohmic spectral 
#densities.
subindices = np.where(Ytrain[:,0] == 1)[0]
ohmicindices = np.where(Ytrain[:,1] == 1)[0]
superindices = np.where(Ytrain[:,2] == 1)[0]

In [ ]:
#Finding the corresponding trajectories 
xsub = Xtrain[subindices,:]
xohmic = Xtrain[ohmicindices,:]
xsuper = Xtrain[superindices,:]

In [ ]:
#Defining arrays to store the variance of the time points in the total trainingset, and the variance of the time 
#points for each class in the training set. We also define an array to store the mean of the class variances for 
#each time point. 
variancetot = np.zeros(Xtrain.shape[1])
variancesub = np.zeros(Xtrain.shape[1])
varianceohmic = np.zeros(Xtrain.shape[1])
variancesuper = np.zeros(Xtrain.shape[1])
variancemean = np.zeros(X.shape[1])

#Calculating the variances 
for i in range(X.shape[1]):
    variancetot[i] = np.std(Xtrain[:,i])**2
    variancesub[i] = np.std(xsub[:,i])**2
    varianceohmic[i] = np.std(xohmic[:,i])**2
    variancesuper[i] = np.std(xsuper[:,i])**2
    variancemean[i] = (varianceohmic[i] + variancesub[i] + variancesuper[i])/3

In [ ]:
#Defining an array to store the difference between the mean of the class variances and the total variance
variancediff = np.zeros(X.shape[1])

#calculating the difference between the mean of the class variances and the total variance for each feature
for i in range(X.shape[1]):
    variancediff[i] = variancetot[i] - variancemean[i]

In [ ]:
#defining a list to the store the indices to be removed if we are to maintain a high classification accuracy. The 
#time point corresponding to the first index in the list is to be removed first, and the time point corresponding 
#to the second index is to be removed next, and so on.
removeorder = []

#For each of the sorted indices of the correlation matrix
for i in range(len(sortedindices)):
    truthval = sortedindices[i][0] in removeorder or sortedindices[i][1] in removeorder #will return true if either index is already in the array
    #if neither index is already in the list, and the indices are not equal
    if truthval == False and sortedindices[i][0] != sortedindices[i][1]:
        #if the difference between the total variance and the mean of the class variances for the first index in 
        #the tuple is less than that for the second index in the tuple 
        if variancediff[sortedindices[i][0]] < variancediff[sortedindices[i][1]]:
            #add the first index to the list
            removeorder.append(sortedindices[i][0])
        else:
            #otherwise add the second
            removeorder.append(sortedindices[i][1])

In [ ]:
#Finding the last index to be removed
for i in range(X.shape[1]):
    if len(np.where(np.array(removeorder)==i)[0]) == 0:
        lastindex = i

#adding it to the list        
removeorder.append(lastindex)

In [ ]:
#defining an array containing the number of points to be used in descending order. The accuracy and loss 
#is to be evaluated when the number of time points in the trajectories is given by each element in the array.
npoints = np.array([400, 200, 134, 100, 40, 27, 20, 16, 14, 12, 10, 9, 8, 7, 6, 5, 4, 2, 1])

In [ ]:
#arrays to store the final test accuracies
testaccuracy = np.zeros(len(npoints))

#arrays to store the final test loss
testloss = np.zeros(len(npoints))

In [ ]:
for l in range(len(npoints)):
    
    #Finding the indices of the time points to keep in the trajectories 
    indicestokeep = removeorder[len(removeorder)-npoints[l]:len(removeorder)]
    
    #defining the data set
    data = X[:, np.sort(np.array(indicestokeep))]
    
    #defining the corresponding times
    times = t[np.sort(np.array(indicestokeep))]
    
    #Generating the training, test and validation set obtained using the non-uniform discrete Fourier transform
    Xtrain, Xval, Xtest, Ytrain, Yval, Ytest = nufouriertrainvaltest(data, labels, times)  

    #Defining the model
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(250, input_dim = (Xtrain.shape[1]), activation='sigmoid'))
    model.add(tf.keras.layers.Dense(80 , activation = 'sigmoid'))
    model.add(tf.keras.layers.Dense(3, activation = 'softmax'))

    #setting the optimiser equal to the Adam optimiser with learning rate = 0.0001
    opt = tf.keras.optimizers.Adam(learning_rate = 0.0001)

    #compiling the model
    model.compile(optimizer=opt,
                loss = 'categorical_crossentropy',
                metrics = ['categorical_accuracy'])

    #training the model
    history = model.fit(Xtrain, Ytrain, epochs = 10000, validation_data = (Xval, Yval), batch_size = Xtrain.shape[0], verbose=0)
     
    #find the training, validation and test accuracies
    testaccuracy[l] = model.evaluate(Xtest, Ytest)[1]
    
    #finding the loss evaluated on the training, validation and test set
    testloss[l] = model.evaluate(Xtest, Ytest)[0]